In [125]:
from datetime import datetime
import dateparser
import pandas as pd
import numpy as np
import re
import nltk

In [6]:
beers = pd.read_csv('../data/beer_info.csv')
reviews = pd.read_csv('../data/beer_reviews.csv')

In [266]:
styles = pd.read_csv('../data/beer_styles.csv',sep='#*#',header=0)
styles = styles.drop(columns='*')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,style,desc
0,German Bock,Bock is a bottom fermenting lager that genera...
1,German Doppelbock,"“Doppel” meaning “double,” this style is a bi..."
2,German Eisbock,Eisbock is an extremely strong beer with a ty...
3,German Maibock,Also called “Heller Bock” (meaning “Pale Bock...
4,German Weizenbock,The German-style Weizenbock is a wheat versio...
5,American Brown Ale,"Roasted malt, caramel-like and chocolate-like..."
6,English Brown Ale,"Spawned from the Mild Ale tradition, English ..."
7,English Dark Mild Ale,"The quintessential British session beer, like..."
8,German Altbier,"A Düsseldorf specialty, an Altbier is a Germa..."
9,American Black Ale,The American black ale is characterized by th...


In [7]:
beers.set_index('beer_id')
reviews.set_index('reviews_id')

,BAscore,abv,beer_id,beer_img,beer_name,beer_style,brewery,desc,num_reviews,ranking
0,4.35,7.0,1,https://cdn.beeradvocate.com/im/beers/141544.jpg,Yakima Fresh Hop,American IPA,Columbus Brewing Company,None provided.,29,1033
1,3.59,7.0,2,https://cdn.beeradvocate.com/im/beers/94770.jpg,Red Clay IPA,American IPA,Sunken City Brewing Company,None provided.,24,38065
2,3.97,7.0,3,https://cdn.beeradvocate.com/im/beers/118069.jpg,Traitor IPA,American IPA,Badger Hill Brewing Company,None provided.,18,11638
3,3.97,6.5,4,https://cdn.beeradvocate.com/im/beers/93370.jpg,OG Wheat India Pale Ale,American IPA,10 Barrel Brewing Co.,None provided.,22,11567
4,3.96,6.0,5,https://cdn.beeradvocate.com/im/beers/180860.jpg,Streaker,American IPA,Rhinegeist Brewery,"The Base Malt, coming in at 27% of the votes, ...",24,12283


In [9]:
#Clean Ratings
# if string characters appears, than user did not give a ratings breakdown. so clear the line.
r = re.compile('characters')
ratings = reviews.ratings
ratings = ['' if r.search(rating) else rating for rating in ratings]
r = re.compile('look|smell|taste|feel|overall|: ')
ratings = [r.sub('',rating).split(' | ') for rating in ratings]
ratings = [ra if ra == [''] else list(map(float, ra)) for ra in ratings]
reviews.ratings = ratings

In [108]:
# Clean posted date
posted = reviews.posted
def parse_dates(text, origin):
    #Parses absolute and relative dates    
    try:
        date = datetime.strptime(text, '%b %d, %Y')
    except ValueError:
        date = dateparser.parse(text, settings={'RELATIVE_BASE': origin})
    return date
scraping_date = dateparser.parse('Jul 3, 2019')
post_cleaned = [parse_dates(post, scraping_date) for post in posted]
reviews.posted = post_cleaned

In [169]:
# Clean text data
rev_text = reviews.text

In [174]:
rev_text.apply(lambda x: str(x))
_.apply(lambda x: x.lower())
_.apply(lambda x: re.sub('\s+', ' ', x))
_.apply(lambda x: re.sub('[^\w\s]', '', x))
rev_text = _
reviews_df = pd.DataFrame({'id': pd.Series(reviews.beer_id),
                          'text':pd.Series(rev_text)})

0          growler fill from the ohio taproom. take advan...
1          l: gold, transparent. gorgeous head with nice ...
2          this is a good fresh hop ale! great dank taste...
3          bottle dated 10/17/16, poured into tulip. mild...
4          12 ounce bottle into tulip glass, bottled on 1...
5          appearance: pours clear and amber, with a thre...
6          clear sparkling gold with a large fluffy white...
7          pours a dark orange color with decent clarity....
8          12 fluid oz bottle served in a tulip glass thi...
9          half-growler filled at rozi's wine house in la...
10         12oz bottle dated 10/12/18 poured into a teku ...
11         12oz stubby big ol' thanks to dave for a coupl...
12         yakima fresh is a bright and clear golden yell...
13         12 oz bottle poured into spiegelau glass bottl...
14         .the aroma just jumps out of the glass. you ca...
15         bottled 10/09. received in trade with an ohioa...
16         i got this ip

In [224]:
style_lookup = pd.DataFrame({'id':beers.beer_id, 'style':beers.beer_style})
reviews_df = pd.merge(reviews_df, style_lookup, how='left',left_on='id',right_on='id')

In [251]:
reviews_df.shape

(1757044, 3)

In [274]:
beers.brewery.unique()[0]

'Columbus Brewing Company'